# Memory 
<p style="color:green"> When interacting with model, how to keep model aware of past conversations ? </p>
memory class keeps past conversions.


In [2]:
# ChatMessageHistory 
# Use Case: Save message chat, for future reference
from langchain.memory import ChatMessageHistory

history=ChatMessageHistory()
history.add_user_message("Hi model")
history.add_ai_message("What's up?")
print(history)
print("-"*10)
print(history.messages)

messages=[HumanMessage(content='Hi model', additional_kwargs={}, example=False), AIMessage(content="What's up?", additional_kwargs={}, example=False)]
----------
[HumanMessage(content='Hi model', additional_kwargs={}, example=False), AIMessage(content="What's up?", additional_kwargs={}, example=False)]


In [18]:
# Wrapper on ChatMessageHistory, for easy access to variables
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory()
memory.chat_memory.add_user_message("Hi")
memory.chat_memory.add_ai_message("What's up")
#memory.chat_memory.add_message("No label message") , throws erros, seems like an open issue
print(memory.load_memory_variables({}))
print(memory.load_memory_variables({})['history'])

# convert messages to dictionary
from langchain.schema import messages_from_dict, messages_to_dict
messages_to_dict(history.messages)

{'history': "Human: Hi\nAI: What's up"}
Human: Hi
AI: What's up


[{'type': 'human',
  'data': {'content': 'Hi model', 'additional_kwargs': {}, 'example': False}},
 {'type': 'ai',
  'data': {'content': "What's up?",
   'additional_kwargs': {},
   'example': False}}]

In [22]:
# add memory to LLMChain
from langchain import LLMChain,PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI

# Initialize OpenAI Model
import os,configparser

config = configparser.ConfigParser()
config.read('secret.ini') 
''' secret.ini content is following
[OpenAI]
openai_api_key = your_Open_AI_key 

'''
os.environ["OPENAI_API_KEY"] = config['OpenAI']['openai_api_key'] # set open AI key as environment variable


template=""" You are a chatbot having conversation with 5 year human child,
{chat_history}
Kid: {request_instruction} 
chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "request_instruction"], template=template
)

memory = ConversationBufferMemory(memory_key="chat_history")

# Build chain
llm_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=memory,
)
lang=input("Language?")
request_instruction=f"Good Morning in {lang}"

llm_chain.predict(request_instruction=request_instruction)




> Entering new  chain...
Prompt after formatting:
 You are a chatbot having conversation with 5 year human child,

Kid: Good Morning in Hindi 
chatbot:

> Finished chain.


' Namaste! Kaise hain aap?'

In [23]:
lang=input("Language?")
request_instruction=f"Good Morning in {lang}"

llm_chain.predict(request_instruction=request_instruction)



> Entering new  chain...
Prompt after formatting:
 You are a chatbot having conversation with 5 year human child,
Human: Good Morning in Hindi
AI:  Namaste! Kaise hain aap?
Kid: Good Morning in garhwali 
chatbot:

> Finished chain.


' Swagat nai! Aap kaise hain?'

In [24]:
lang=input("Language?")
request_instruction=f"Good Morning in {lang}" 

llm_chain.predict(request_instruction=request_instruction)



> Entering new  chain...
Prompt after formatting:
 You are a chatbot having conversation with 5 year human child,
Human: Good Morning in Hindi
AI:  Namaste! Kaise hain aap?
Human: Good Morning in garhwali
AI:  Swagat nai! Aap kaise hain?
Kid: Good Morning in French 
chatbot:

> Finished chain.


' Bonjour! Comment allez-vous?'

In [29]:
llm_chain.predict(request_instruction="tell me in english, what was the first message from me? ") 



> Entering new  chain...
Prompt after formatting:
 You are a chatbot having conversation with 5 year human child,
Human: Good Morning in Hindi
AI:  Namaste! Kaise hain aap?
Human: Good Morning in garhwali
AI:  Swagat nai! Aap kaise hain?
Human: Good Morning in French
AI:  Bonjour! Comment allez-vous?
Human: In how many languages you have converted?
AI:  Main saat bhashaon mein apna bhashantar kar sakta hoon - Hindi, Garhwali, French, Spanish, Portuguese, German, aur Japanese.
Human: what was the first message from me?
AI:  Aapne pehla message ka Hindi mein tha - "Namaste! Kaise hain aap?"
Human: what was the first message from me? in english
AI:  Aapne pehla message English mein tha - "Good Morning in Hindi"
Human: in english, what was the first message from me? 
AI:  Aapne pehla message English mein tha - "Good Morning in Hindi."
Kid: tell me in english, what was the first message from me?  
chatbot:

> Finished chain.


' Aapne pehla message English mein tha - "Good Morning in Hindi."'

In [31]:
# load a pdf and multiple input
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/lobel-frog-toad.pdf")
pages = loader.load_and_split()

# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma # vector storage

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(pages, embedding_function)

# retriever
retriever = db.as_retriever()


In [44]:
# Stackoverflow question:
#https://stackoverflow.com/questions/76692869/how-to-add-memory-to-load-qa-chain-or-how-to-implement-conversationalretrievalch?noredirect=1#comment135219682_76692869
# doc string prompt
#
prompt_template = """You are a Chat customer support agent.\
        Address the customer as Dear Mr. or Miss. depending on customer's gender followed by Customer's First Name.\
        Use the following pieces of context to answer the question at the end.\
        If you don't know the answer, just say that you don't know, don't try to make up an answer.\
        Below are the details of the customer:\
        Customer's Name : {Customer_Name} \
        Customer's Resident State: {Customer_State}\
        Customer's Gender: {Customer_Gender}\
        {context}\
        Question: {question}\
        Answer: """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question","Customer_Name","Customer_State","Customer_Gender"]
)
from langchain.chains.question_answering import load_qa_chain

memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")
chain = load_qa_chain(
    OpenAI(temperature=0), chain_type="stuff", memory=memory, prompt=PROMPT
)


query="why frog sent letter to Todd??"
docs=db.similarity_search(query=query)

# building the dictionary for chain

chain_input={
    "input_documents": docs,
    "context":"This is contextless",
    "question":query,
    "Customer_Name":"Bob",
    "Customer_State":"NY",
    "Customer_Gender":"Male"
}

result=chain(chain_input, return_only_outputs=True)

In [43]:
result['output_text']

' Dear Mr. Bob, Frog sent a letter to Toad because he wanted to show Toad that he was glad to be his best friend. He wrote in the letter, "Dear Toad, I am glad that you are my best friend. Your best friend, Frog."'